In [ ]:
pip install pdfminer

In [ ]:
pip install docx2txt

In [4]:
# Import libaries
import regex as re
import pdfminer
from io import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import os
import docx2txt

In [6]:
# Uploading the pdf resume and reading resume  
rootdir = '/content/drive/MyDrive/Data Science/FullThrottle_lab'
files = os.listdir(rootdir)
#print(files)
file_names = []
for file in os.listdir(rootdir):
    #print(os.path.join( file))
    file_names.append(os.path.join( '/content/drive/MyDrive/Data Science/FullThrottle_lab',file))
file_names

['/content/drive/MyDrive/Data Science/FullThrottle_lab/resume_6.pdf',
 '/content/drive/MyDrive/Data Science/FullThrottle_lab/resume_17.pdf',
 '/content/drive/MyDrive/Data Science/FullThrottle_lab/resume_5.pdf',
 '/content/drive/MyDrive/Data Science/FullThrottle_lab/resume_3.pdf',
 '/content/drive/MyDrive/Data Science/FullThrottle_lab/resume_10.pdf',
 '/content/drive/MyDrive/Data Science/FullThrottle_lab/resume_15.pdf',
 '/content/drive/MyDrive/Data Science/FullThrottle_lab/resume_7.pdf',
 '/content/drive/MyDrive/Data Science/FullThrottle_lab/resume_18.pdf',
 '/content/drive/MyDrive/Data Science/FullThrottle_lab/resume_16.pdf',
 '/content/drive/MyDrive/Data Science/FullThrottle_lab/resume_11.pdf',
 '/content/drive/MyDrive/Data Science/FullThrottle_lab/resume_2.pdf',
 '/content/drive/MyDrive/Data Science/FullThrottle_lab/resume_1.pdf',
 '/content/drive/MyDrive/Data Science/FullThrottle_lab/resume_19.pdf',
 '/content/drive/MyDrive/Data Science/FullThrottle_lab/resume_4.pdf',
 '/content/dr

In [7]:
# Split all the resume line based on reqruirements
def open_pdf_file(file_name):
    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    pagenums = set()
    infile = open(file_name, 'rb')
    for page in PDFPage.get_pages(infile, pagenums):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close()

    result = []

    for line in text.split('\n'):
        line2 = line.strip()
        if line2 != '':
            result.append(line2)
    return (result)

In [8]:
# Converting the resume to text 
def open_docx_file(file_name):
    temp = docx2txt.process(file_names[0])
    text = [line.replace('\t', ' ') for line in temp.split('\n') if line]
    text = [t for t in text if len(t) > 1]
    return (text)

In [9]:
# Removing Punctuations from resume 
def remove_punctuations(line):
    return re.sub(r'(\.|\,)', '', line)

def preprocess_document(document):
    for index, line in enumerate(document):
        line = line.lower()
        line = remove_punctuations(line)
        
        line = line.split(' ')
        while '' in line:
            line.remove('')
            
        while ' '  in line:
            line.remove(' ')
            
            
        document[index] = ' '.join(line)
    return (document)

In [10]:
# Extracting few more information from resume(phone_no)
def get_phone_no(document):
    #This function has to be further modified better and accurate results.
    #Possible phone number formats - Including +91 or just with the numbers.
    
    mob_num_regex = r'''(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)
                        [-\.\s]*\d{3}[-\.\s]??\d{4}|\d{5}[-\.\s]??\d{4})'''
    pattern = re.compile(mob_num_regex)
    matches = []
    for line in document:
        match = pattern.findall(line)
        for mat in match:
            if len(mat) > 9:
                matches.append(mat)

    return (matches)

In [11]:
# Extracting education information from resume:
def get_education(document):
    education_terms = []
    with open('/content/drive/MyDrive/Data Science/FullThrottle_lab/education.txt', 'r') as file:
        education_terms = file.readlines()
    
    education_terms = [term.strip('\n') for term in education_terms]
    education = []
    for line in document:
        for word in line.split(' '):
            if len(word) > 2 and word in education_terms:
                if line not in education:
                    education.append(line)
    #print (education)
    return (education)

In [12]:
## Extracting Skill information from resume:
def get_skills(document):
    skill_terms = []
    with open('/content/drive/MyDrive/Data Science/FullThrottle_lab/valid_skills.txt', 'r') as file:
        skill_terms = file.readlines()
    
    skill_terms = [term.strip('\n') for term in skill_terms]
    skills = []
    
    for line in document:
        words = line.split(' ')
        
        for word in words:
            if word in skill_terms:
                if word not in skills:
                    skills.append(word)
                    
        word_pairs = []
        for i in zip(words[:-1], words[1:]):
            word_pairs.append(i[0] + ' ' + i[1])   #This is to find skills like 'data science' i.e skills containint two words.    return (skills)
            
        for pair in word_pairs:
            if pair in skill_terms:
                if pair not in skills:
                    skills.append(pair)
                    
    return (skills)

In [13]:
# Extracting experience information from resume:
def get_experience(document):
    pattern1 = re.compile(r'(jan(uary)?|feb(ruary)?|mar(ch)?|apr(il)?|may|jun(e)?|jul(y)?|aug(ust)?|sep(tember)?|oct(ober)?|nov(ember)?|dec(ember)?)(\s|\S)(\d{2,4}).*(jan(uary)?|feb(ruary)?|mar(ch)?|apr(il)?|may|jun(e)?|jul(y)?|aug(ust)?|sep(tember)?|oct(ober)?|nov(ember)?|dec(ember)?)(\s|\S)(\d{2,4})')
    pattern2 = re.compile(r'(\d{2}(.|..)\d{4}).{1,4}(\d{2}(.|..)\d{4})')
    pattern3 = re.compile(r'(\d{2}(.|..)\d{4}).{1,4}(present)')
    pattern4 = re.compile(r'(jan(uary)?|feb(ruary)?|mar(ch)?|apr(il)?|may|jun(e)?|jul(y)?|aug(ust)?|sep(tember)?|oct(ober)?|nov(ember)?|dec(ember)?)(\s|\S)(\d{2,4}).*(present)')
    patterns = [pattern1, pattern2, pattern3, pattern4]
    experience = []
    for index, line in enumerate(document):
        for pattern in patterns:
            exp = pattern.findall(line)
            if len(exp) > 0:
                experience.append(document[index:index+4])
                
    return (experience)

In [15]:
# Combining all the information from resume 
phone_nos = []
education_1 = []
education_2 = []
skills_1 = []
skills_2 = []
experience_1 = []
experience_2 = []

for file_name in file_names:
    print(file_name)
    if file_name.endswith('.pdf'):
        document = open_pdf_file(file_name)
    
    
    # email = get_email(document)
    phone_no = get_phone_no(document)
    document = preprocess_document(document)
    #print ('\n\n')
    #print (file_name)
    #print ('Email is {} phone number is {}'.format(email, phone_no))
        
    if len(phone_no) > 0:
        phone_nos.append(phone_no[0])
    else:
        phone_nos.append('')
    
    education = get_education(document)
    #print ('Education ', get_education(document))
    if len(education) > 1:
        education_1.append(education[0])
        education_2.append(education[1])
    elif len(education) == 1:
        education_1.append(education[0])
        education_2.append('')
    elif len(education) == 0:
        education_1.append('')
        education_2.append('')
        
    skills = get_skills(document)
    #print ('Skills ', skills)
    
    if len(skills) > 1:
        skills_1.append(skills[0])
        skills_2.append(skills[1])
    elif len(skills) == 1:
        skills_1.append(skills[0])
        skills_2.append('')
    elif len(skills) == 0:
        skills_1.append('')
        skills_2.append('')
        
    experience = get_experience(document)
    #print ('Experience ', get_experience(document))
    if len(experience) > 1:
        experience_1.append(experience[0])
        experience_2.append(experience[1])
    elif len(experience) == 1:
        experience_1.append(experience[0])
        experience_2.append('')
    elif len(experience) == 0:
        experience_1.append('')
        experience_2.append('')

/content/drive/MyDrive/Data Science/FullThrottle_lab/resume_6.pdf
/content/drive/MyDrive/Data Science/FullThrottle_lab/resume_17.pdf
/content/drive/MyDrive/Data Science/FullThrottle_lab/resume_5.pdf
/content/drive/MyDrive/Data Science/FullThrottle_lab/resume_3.pdf
/content/drive/MyDrive/Data Science/FullThrottle_lab/resume_10.pdf
/content/drive/MyDrive/Data Science/FullThrottle_lab/resume_15.pdf
/content/drive/MyDrive/Data Science/FullThrottle_lab/resume_7.pdf
/content/drive/MyDrive/Data Science/FullThrottle_lab/resume_18.pdf
/content/drive/MyDrive/Data Science/FullThrottle_lab/resume_16.pdf
/content/drive/MyDrive/Data Science/FullThrottle_lab/resume_11.pdf
/content/drive/MyDrive/Data Science/FullThrottle_lab/resume_2.pdf
/content/drive/MyDrive/Data Science/FullThrottle_lab/resume_1.pdf
/content/drive/MyDrive/Data Science/FullThrottle_lab/resume_19.pdf
/content/drive/MyDrive/Data Science/FullThrottle_lab/resume_4.pdf
/content/drive/MyDrive/Data Science/FullThrottle_lab/resume_8.pdf
/co

In [16]:
# Making table by using pandas DataFrame
import pandas as pd
df = pd.DataFrame({'phone nos':phone_nos, 'education 1':education_1, 'education 2':education_2, 'skills 1':skills_1, 
              'skills 2':skills_2, 'experience 1':experience_1, 'experiece 2':experience_2})

In [20]:
df.to_excel('/content/drive/MyDrive/Data Science/FullThrottle_lab/extracted_data.xlsx',index=False)

In [21]:
df

,phone nos,education 1,education 2,skills 1,skills 2,experience 1,experiece 2
0,6362090529,kristu jayanti college,,,,"[05/2018 – present, achievements/tasks, 636209...","[02/2017 – 08/2017, achievements/tasks, i work..."
1,,✦ made a fully functional university education...,btech computer science and engineering,python,javascript,[higher secondary education cbse 04/2015-04/20...,
2,82108-3827,bachelor of technology (b tech) computer scien...,science web scraping web crawling,python,c++,,
3,73378 3406,of engineering after my high school graduation...,saxion university of applied sciences,,,"[aug 16 - feb 17 research internship, research...",
4,9183447743,,,,,"[(cid:17) july2016–may2020, python&rprogrammin...",
5,7666658007,university of mumbai mumbai india,bachelor of engineering in computer engineerin...,c,javascript,"[aug 2016 – dec 2016, • poc chat bot (using wi...","[dec 2016 - present, • my responsibilities wer..."
6,,,,c,,,
7,,bachelor of science,,c,,,
8,9008036407,amrita school of engineering - bengaluru karna...,svei composite pu college - mysore karnataka,css,html,"[june 2016 to february 2019, education, b tech...",
9,8087789641,with three year of experience encompassing ski...,science field,data science,python,"[dec 2017 – present, freelancing:, freelancing...","[sep 2016 – nov 2017, sep 2015 – sep 2016, aca..."
